# 한총 계절 장바구니 분석

In [1]:
#데이터셋 가져오기
#order 컬럼명 바꾸기 - 숫자로 되어 있는 부분에 Order (purchase event) 입력
import pandas as pd
#카테고리 전처리
import re
# 장바구니 분석
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import apriori, association_rules
import datetime as dt
import os

### 0. 경로 설정

In [2]:
input_path = 'C:/GMG_R&D/3. 장바구니 분석/1. 한총/계절' #input("작업 파일 경로를 입력하세요")
os.chdir(input_path)
path=os.getcwd()
print(path)

#폴더 생성
#x = dt.datetime.now()

file_name=f'계절분석결과'

if not os.path.exists(file_name):
    os.mkdir(file_name) 

C:\GMG_R&D\3. 장바구니 분석\1. 한총\계절


In [3]:


def frozen_convert(sets) : 
        return [list(sets)]


### 1. 한총 데이터 로드

In [4]:
df= pd.read_clipboard(sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42812 entries, Model Code (v41) to OTTA0002,OTTA0005,sam20220221004,sam20220221006
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   2019 겨울(12-2)   42812 non-null  int64
 1   2020년 봄(3-5)    42812 non-null  int64
 2   2020년 여름(6-8)   42812 non-null  int64
 3   2020년 가을(9-11)  42812 non-null  int64
 4   2020년 겨울(12-2)  42812 non-null  int64
 5   2021년 봄(3-5)    42812 non-null  int64
 6   2021년 여름(6-8)   42812 non-null  int64
 7   2021년 가을(9-11)  42812 non-null  int64
 8   2021년 겨울(12-2)  42812 non-null  int64
 9   2022년 봄(3-5)    42812 non-null  int64
 10  2022년 여름(6-8)   42812 non-null  int64
dtypes: int64(11)
memory usage: 3.9+ MB


In [5]:
df=df[1:]
df=df.reset_index().rename(columns={"index": "Model Code (v41)"})
df

,Model Code (v41),2019 겨울(12-2),2020년 봄(3-5),2020년 여름(6-8),2020년 가을(9-11),2020년 겨울(12-2),2021년 봄(3-5),2021년 여름(6-8),2021년 가을(9-11),2021년 겨울(12-2),2022년 봄(3-5),2022년 여름(6-8)
0,EP-DG930MBKGKR,1596,4652,1390,1780,1174,1348,1320,1968,1412,1794,562
1,AA-BC7AU5B,2396,4477,2445,47,0,0,0,0,0,0,0
2,MD-BTPENPLUS,0,3399,0,0,0,0,0,0,0,0,0
3,GP-TOG977MOETK,0,2550,0,0,0,0,0,0,0,0,0
4,AKGN400BLK,0,2538,132,47,13,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
42806,"SM-R860NZKAKOO,SM-R860NZKAKOO,EP-TA800XWKGKR",0,0,0,0,0,0,0,0,0,0,1
42807,SAM20220503005,0,0,0,0,0,0,0,0,0,0,1
42808,"F400922,F400922,F400922,F400922,F400922,F400922",0,0,0,0,0,0,0,0,0,0,1
42809,"RF60B91U2W6,WF21T9500KW7T,KQ75QB65-W1B5,DW60T7...",0,0,0,0,0,0,0,0,0,0,1


### 2. 카테고리 파일 로드

In [6]:
#카테고리 가져오기
rex= pd.read_clipboard(sep='\t')
rex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Category2D        37 non-null     object
 1   Category2D_Value  37 non-null     object
dtypes: object(2)
memory usage: 720.0+ bytes


### 계절별 비중

In [8]:

for season in df.columns[1:]:
    print('****************************')
    print(season )
    all=df[season].sum()
    print('전체: ',all)
    print('2개이상: ',df[df[['Model Code (v41)',season]]['Model Code (v41)'].str.contains(',')][season].sum())
    
    print("제품 2개 구매 비중: ",round((df[df[['Model Code (v41)',season]]['Model Code (v41)'].str.contains(',')][season].sum()/all)*100,2))

    # 데이터 만들기
    season_df=df[df[['Model Code (v41)',season]]['Model Code (v41)'].str.contains(',')]
    season_df=season_df[season_df[season]!= 0]
    print('콤마 & order 0제외 행개수:',season_df.shape[0])
    season_df=season_df.reset_index(drop=True)
    

    #전체 SKU 하나의 리스트로
    a = season_df['Model Code (v41)'].tolist()
    b = ','.join(a) #리스트 문자열로
    c = b.split(',') #문자열 개별 문자로 리스트화 
    model = list(set(c)) #중복 제거

    #전체 SKU 담을 딕셔너리
    dic_category = {}

    for m in model :
        flag = 1
        for i in range(len(rex)):
            r = re.compile(rex.Category2D_Value[i])
            if r.search(m):
                dic_category[m] = rex.Category2D[i]
                flag = 0
                continue
        if flag :
            dic_category[m] = '기타'

    #데이터셋 만들기
    dataset = []    # 데이터셋 구조 [ [a,b], [a,c], [a,b,c] .... ]

    for i in range(len(season_df)) :
        sku = season_df['Model Code (v41)'][i]
        revised = sku.split(",")
        code = []
        for j in range(len(revised)) :
            code.append(dic_category[revised[j]])   #각 SKU 리스트에 담기
        
        for k in range(season_df[season][i]) :   #order 수만큼 담기
            dataset.append(code) 
    

    #기타제거
    ho = []
    rm_set = {'기타'}
    for s in dataset:
        arr_new = [i for i in s if i not in rm_set]
        ho.append(arr_new) # [5, 7, 2, 7]
    dataset=ho

    print('기타 제거 후, 행 수: ',len(dataset))
    
    #장바구니 분석
    te = TransactionEncoder()
    te_ary = te.fit(dataset).transform(dataset) #True / False
    season2 = pd.DataFrame(te_ary, columns=te.columns_)
    itemsets_data = apriori(season2, min_support=0.0001, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
    rules = association_rules(itemsets_data, metric='lift', min_threshold=1).sort_values('lift',ascending=False).reset_index(drop=True)
    rules['count']=(rules['support']*season_df[season].sum()).astype(int)

    x = dt.datetime.now()

    num_conq=1
    #N=4


    for num_ant in [1,2,3]:
        final_season = rules[rules.apply(lambda x:True if len(x.antecedents) == int(num_ant) else False, axis=1) & rules.apply(lambda x:True if len(x.consequents) == int(num_conq) else False, axis=1)]
        final_season = final_season.sort_values('confidence', ascending=False)
        #final_season = final_season.round(N)
        final_season[['antecedents','consequents']] =final_season[['antecedents','consequents']].applymap(lambda x: frozen_convert(x))
        final_season['antecedents']=final_season['antecedents'].astype(str).apply(lambda x: x.replace('[[','['))
        final_season['antecedents']=final_season['antecedents'].apply(lambda x: x.replace(']]',']'))
        final_season['consequents']=final_season['consequents'].astype(str).apply(lambda x: x.replace('[[','['))
        final_season['consequents']=final_season['consequents'].apply(lambda x: x.replace(']]',']'))
        final_season.to_excel(f'{path}/{file_name}/한총 {season} 분석결과파일_{num_ant}.xlsx')

****************************
2019 겨울(12-2)
전체:  61131
2개이상:  19288
제품 2개 구매 비중:  31.55
콤마 & order 0제외 행개수: 904
****************************
2020년 봄(3-5)
전체:  84317
2개이상:  4462
제품 2개 구매 비중:  5.29
콤마 & order 0제외 행개수: 1983
****************************
2020년 여름(6-8)
전체:  114848
2개이상:  33265
제품 2개 구매 비중:  28.96
콤마 & order 0제외 행개수: 1933
****************************
2020년 가을(9-11)
전체:  83172
2개이상:  5320
제품 2개 구매 비중:  6.4
콤마 & order 0제외 행개수: 2692
****************************
2020년 겨울(12-2)
전체:  192408
2개이상:  47187
제품 2개 구매 비중:  24.52
콤마 & order 0제외 행개수: 2492
****************************
2021년 봄(3-5)
전체:  113521
2개이상:  13150
제품 2개 구매 비중:  11.58
콤마 & order 0제외 행개수: 2813
****************************
2021년 여름(6-8)
전체:  211030
2개이상:  7990
제품 2개 구매 비중:  3.79
콤마 & order 0제외 행개수: 3080
****************************
2021년 가을(9-11)
전체:  203683
2개이상:  11660
제품 2개 구매 비중:  5.72
콤마 & order 0제외 행개수: 4539
****************************
2021년 겨울(12-2)
전체:  186925
2개이상:  12406
제품 2개 구매 비중:  6.64
콤마 & order 0제외 행개수: